# 190323 SparkSQL 실습

In [2]:
sc

Out[ 1 ]: <SparkContext master=local[8] appName=Databricks Shell>

In [3]:
import collections

from pyspark import StorageLevel
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time
from pyspark.sql.functions import udf
from pyspark.sql.types import *

## 데이터 불러오기

이 데이터는 2001년부터 2019년까지의 모든 야구 경기를 나타낸 데이터입니다.

- team1, team2 : 경기에 참여한 팀
- teams : team1 + team2
- stadium : 해당 경기를 치룬 경기장
- etc : 해당 경기의 상태 (제대로 경기를 했는지, 그렇지 못하면 무슨 사유로 취소되었는지)
- year, day_of_week, date : 그 경기가 이루어진 년도, 요일, 날짜에 대한 데이터

In [5]:
stadium = spark.read.csv("/FileStore/tables/stadium.csv", header = True)
stadium.show()

+-----+-----+-------+---+----+-----------+----+---------+
team1|team2|stadium|etc|year|day_of_week|date| teams|
+-----+-----+-------+---+----+-----------+----+---------+
 삼성| NC| 창원| -|2019| 토|3.23| 삼성 NC|
 한화| 두산| 잠실| -|2019| 토|3.23|한화 두산|
 KT| SK| 문학| -|2019| 토|3.23| KT SK|
 LG| KIA| 광주| -|2019| 토|3.23| LG KIA|
 키움| 롯데| 사직| -|2019| 토|3.23|키움 롯데|
 삼성| NC| 창원| -|2019| 일|3.24| 삼성 NC|
 한화| 두산| 잠실| -|2019| 일|3.24|한화 두산|
 KT| SK| 문학| -|2019| 일|3.24| KT SK|
 LG| KIA| 광주| -|2019| 일|3.24| LG KIA|
 키움| 롯데| 사직| -|2019| 일|3.24|키움 롯데|
 KT| NC| 창원| -|2019| 화|3.26| KT NC|
 키움| 두산| 잠실| -|2019| 화|3.26|키움 두산|
 LG| SK| 문학| -|2019| 화|3.26| LG SK|
 한화| KIA| 광주| -|2019| 화|3.26| 한화 KIA|
 삼성| 롯데| 사직| -|2019| 화|3.26|삼성 롯데|
 KT| NC| 창원| -|2019| 수|3.27| KT NC|
 키움| 두산| 잠실| -|2019| 수|3.27|키움 두산|
 LG| SK| 문학| -|2019| 수|3.27| LG SK|
 한화| KIA| 광주| -|2019| 수|3.27| 한화 KIA|
 삼성| 롯데| 사직| -|2019| 수|3.27|삼성 롯데|
+-----+-----+-------+---+----+-----------+----+---------+
only showing top 20 rows

In [6]:
stadium.printSchema()

root
-- team1: string (nullable = true)
-- team2: string (nullable = true)
-- stadium: string (nullable = true)
-- etc: string (nullable = true)
-- year: string (nullable = true)
-- day_of_week: string (nullable = true)
-- date: string (nullable = true)
-- teams: string (nullable = true)

## 야구 팀은 뭐가 있지?... 그 팀은 몇 번 경기를 했을까?
18개의 팀이 나오지만 2001년부터의 데이터가 모여있기 때문에 현재는 없어진 팀들도 있습니다.

현재는 두산, SK, 키움, 롯데, NC, 삼성, KT, KIA, LG, 한화로 10개 팀이 있습니다.

In [8]:
stadium.groupBy('team1').count().show()

+--------+-----+
 team1|count|
+--------+-----+
 키움| 71|
 WEST| 3|
 롯데| 1332|
 현대| 455|
 NC| 515|
 넥센| 688|
 삼성| 1339|
 KT| 389|
 KIA| 1328|
 SK| 1337|
 두산| 1345|
 LG| 1320|
히어로즈| 67|
 한화| 1325|
 EAST| 2|
 드림| 2|
 나눔| 2|
 우리| 63|
+--------+-----+

## Q1. 팀 개수 한 번에 세고 싶다면?
Hint : stadium.select()를 하고 괄호 안에 중복되지 않는 값을 세는 메소드를 넣어줍니다.

+---------------------+
count(DISTINCT team1)|
+---------------------+
 18|
+---------------------+

## Q2. etc에는 무슨 데이터들이 들어가있을까요?

+------------+-----+
 etc|count|
+------------+-----+
 기타| 3|
 우천취소| 546|
 -|11028|
미세먼지취소| 4|
그라운드사정| 2|
+------------+-----+

## 각 년도별 총 경기 횟수

In [14]:
# asc : 오름차순 / desc : 내림차순
stadium.sort(desc('year')).groupBy('year').count().show()

+----+-----+
year|count|
+----+-----+
2019| 715|
2018| 762|
2017| 761|
2016| 784|
2015| 802|
2014| 637|
2013| 635|
2012| 606|
2011| 610|
2010| 595|
2009| 532|
2008| 504|
2007| 504|
2006| 504|
2005| 504|
2004| 532|
2003| 532|
2002| 532|
2001| 532|
+----+-----+

## 2019년 시즌 경기 중 각 경기장에서 일어나는 경기의 수

잠실은 두산과 LG가 같이 홈으로 사용하여 count가 타 구장에 비해 2배 높을 수밖에 없습니다.

In [16]:
stadium.where(stadium['year'] == 2019).sort(asc('stadium')).groupBy('stadium').count().show()

+-------+-----+
stadium|count|
+-------+-----+
 고척| 72|
 광주| 71|
 대구| 72|
 대전| 72|
 문학| 71|
 사직| 71|
 수원| 72|
 잠실| 143|
 창원| 71|
+-------+-----+

## 특정 팀이 참여한 경기만 뽑아보기

각자 원하시는 팀 이름을 넣어주세용

In [18]:
team = stadium.filter("teams like '%두산%'")

In [19]:
team.show()

+-----+-----+-------+---+----+-----------+----+---------+
team1|team2|stadium|etc|year|day_of_week|date| teams|
+-----+-----+-------+---+----+-----------+----+---------+
 한화| 두산| 잠실| -|2019| 토|3.23|한화 두산|
 한화| 두산| 잠실| -|2019| 일|3.24|한화 두산|
 키움| 두산| 잠실| -|2019| 화|3.26|키움 두산|
 키움| 두산| 잠실| -|2019| 수|3.27|키움 두산|
 키움| 두산| 잠실| -|2019| 목|3.28|키움 두산|
 두산| 삼성| 대구| -|2019| 금|3.29|두산 삼성|
 두산| 삼성| 대구| -|2019| 토| 3.3|두산 삼성|
 두산| 삼성| 대구| -|2019| 일|3.31|두산 삼성|
 KT| 두산| 잠실| -|2019| 화|4.02| KT 두산|
 KT| 두산| 잠실| -|2019| 수|4.03| KT 두산|
 KT| 두산| 잠실| -|2019| 목|4.04| KT 두산|
 NC| 두산| 잠실| -|2019| 금|4.05| NC 두산|
 NC| 두산| 잠실| -|2019| 토|4.06| NC 두산|
 NC| 두산| 잠실| -|2019| 일|4.07| NC 두산|
 두산| 롯데| 사직| -|2019| 화|4.09|두산 롯데|
 두산| 롯데| 사직| -|2019| 수| 4.1|두산 롯데|
 두산| 롯데| 사직| -|2019| 목|4.11|두산 롯데|
 두산| LG| 잠실| -|2019| 금|4.12| 두산 LG|
 두산| LG| 잠실| -|2019| 토|4.13| 두산 LG|
 두산| LG| 잠실| -|2019| 일|4.14| 두산 LG|
+-----+-----+-------+---+----+-----------+----+---------+
only showing top 20 rows

## Q3. 그 팀이 2018년에 각각 어느 경기장에서 몇 번을 경기했을까요? (내림차순 정렬)
2018년 총 경기 횟수 144회

한 시즌에 A 팀이 B 팀과 경기하는 횟수는 보통 16회

따라서 홈 구장을 제외하고 타 구장에선 보통 8번 정도 경기를 하게 되고, 나머지는 모두 홈 구장에서 경기가 이루어집니다.

+-------+-----+
stadium|count|
+-------+-----+
 잠실| 87|
 사직| 10|
 대전| 8|
 수원| 8|
 문학| 8|
 대구| 8|
 마산| 8|
 고척| 8|
 광주| 8|
+-------+-----+

## Q4. 미세먼지취소였던 날의 경기를 뽑아주세요

+-----+-----+-------+------------+----+-----------+----+--------+
team1|team2|stadium| etc|year|day_of_week|date| teams|
+-----+-----+-------+------------+----+-----------+----+--------+
 NC| 두산| 잠실|미세먼지취소|2018| 금|4.06| NC 두산|
 삼성| SK| 문학|미세먼지취소|2018| 금|4.06| 삼성 SK|
 한화| KT| 수원|미세먼지취소|2018| 금|4.06| 한화 KT|
 롯데| KIA| 광주|미세먼지취소|2018| 일|4.15|롯데 KIA|
+-----+-----+-------+------------+----+-----------+----+--------+